In [1]:
import pandas as pd
import os

# --- Configuration ---
file_names = {
    "ps_BART.xlsx": "ps-BART 200 trees",
    "ps_BART_Trees_50.xlsx": "ps_BART_Trees_50",
    "ps_BART_Trees_100.xlsx": "ps_BART_Trees_100",
    "ps_BART_Trees_300.xlsx": "ps_BART_Trees_300"
}

# Dictionary to hold the results
# Structure: { 'ColumnName': { 'Mean': { 'FileName': val }, 'StdDev': { 'FileName': val } } }
results_data = {}

# --- Step 1 & 2: Read Files and Calculate Stats ---
print("--- Processing Excel Files ---")

for filename, pretty_name in file_names.items():
    if os.path.exists(filename):
        print(f"Processing: {filename} as '{pretty_name}'")
        try:
            # Read the Excel file
            df = pd.read_excel(filename)
            
            # Select only numeric columns to avoid errors with text columns
            numeric_df = df.select_dtypes(include=['number'])
            
            # Calculate mean and std
            # Resulting 'stats' has 'mean' and 'std' as ROWS (index), and columns are the variables
            stats = numeric_df.agg(['mean', 'std'])
            
            # Iterate through the columns present in this file's stats
            for col in stats.columns:
                # Initialize the dictionary for this column if it doesn't exist yet
                if col not in results_data:
                    results_data[col] = {'Mean': {}, 'StdDev': {}}
                
                # --- CORRECTED EXTRACTION LOGIC ---
                # We use .loc[row_label, column_label]
                # We assume the index contains 'mean' and 'std'
                
                if 'mean' in stats.index:
                    val = stats.loc['mean', col]
                    results_data[col]['Mean'][pretty_name] = val
                    
                if 'std' in stats.index:
                    val = stats.loc['std', col]
                    results_data[col]['StdDev'][pretty_name] = val
                    
        except Exception as e:
            print(f"Error reading {filename}: {e}")
    else:
        print(f"Warning: File not found: {filename}")

# --- Step 3: Create the Final Consolidated Table ---
if not results_data:
    print("No data processed.")
else:
    print("\n--- Creating Final Consolidated Table ---")
    
    final_rows = []
    
    # Sort columns alphabetically for cleaner output, or remove sorted() to keep insertion order
    for col_name in results_data.keys():
        stats_dict = results_data[col_name]
        
        # Create the mean row
        mean_row = {'Statistic': f'{col_name} (Mean)'}
        # Create the std dev row
        std_row = {'Statistic': f'{col_name} (Std Dev)'}
        
        # Fill in data for each file (column in final table)
        for pretty_name in file_names.values():
            mean_row[pretty_name] = stats_dict['Mean'].get(pretty_name, 'N/A')
            std_row[pretty_name] = stats_dict['StdDev'].get(pretty_name, 'N/A')

        final_rows.append(mean_row)
        final_rows.append(std_row)

    # Create DataFrame
    final_df = pd.DataFrame(final_rows)
    
    # Ensure columns are in the correct order (Statistic first, then the file names)
    cols = ['Statistic'] + list(file_names.values())
    final_df = final_df[cols]
    
    # --- Step 4: Display and Save ---
    print("\n==================================================")
    print(final_df.to_string())
    print("==================================================")
    
    output_filename = "comparison_statistics.xlsx"
    final_df.to_excel(output_filename, index=False)
    print(f"\nSaved to {output_filename}")

--- Processing Excel Files ---
Processing: ps_BART.xlsx as 'ps-BART 200 trees'
Processing: ps_BART_Trees_50.xlsx as 'ps_BART_Trees_50'
Processing: ps_BART_Trees_100.xlsx as 'ps_BART_Trees_100'
Processing: ps_BART_Trees_300.xlsx as 'ps_BART_Trees_300'

--- Creating Final Consolidated Table ---

                           Statistic  ps-BART 200 trees  ps_BART_Trees_50  ps_BART_Trees_100  ps_BART_Trees_300
0          empirical_ATE_RMSE (Mean)           0.301437          0.300400           0.303325           0.304817
1       empirical_ATE_RMSE (Std Dev)           0.158172          0.154608           0.148142           0.151365
2           empirical_ATE_MAE (Mean)           0.256511          0.255381           0.257435           0.259374
3        empirical_ATE_MAE (Std Dev)           0.163112          0.157669           0.152430           0.155551
4          empirical_ATE_MAPE (Mean)           0.159919          0.159089           0.160486           0.161403
5       empirical_ATE_MAPE (Std D